In [1]:
from IPython.display import display, HTML
import scipy.constants as sc
from scipy.optimize import fsolve
from ipywidgets import interact, widgets
%matplotlib notebook

import numpy as np
import cmath
import matplotlib.pyplot as plt
import scipy.constants as sc
plt.rcParams.update({
    "text.usetex": True,
    'text.latex.preamble': r"\usepackage{amsmath} \boldmath"
})
import os
import h5py

print(os.getpid())
%cd ../

12256
/media/work/docs/codes/QuCF/scripts-py


In [2]:
# --- Launch to change cells' width ---
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pylib.mix as mix
import pylib.qucf_read as qucf_r
import pylib.qucf_oracle as qucf_o
path_ = "../simulations/tests/test-diag-matrix"
file_name_oracle_ = "circuit_OH"

In [4]:
# ---------------------------------------------------------------
# --- Read the matrix ---
# ---------------------------------------------------------------
mix.reload_module(qucf_r)

# nr = 4, 1
# nr, nf = 4, 1
nr, nf = 3, 2
N = 1 << (nr + nf)
D = np.identity(N)

In [5]:
# --------------------------------------------------------------
# --- Chose the case ---
# --------------------------------------------------------------
mix.reload_module(mix)
mix.reload_module(qucf_o)
A = np.zeros((N,N), dtype=complex)

a = 1.2
b = 10.4j
c = 4.6 + 0.1j
d = 4.9 + 2.3j


# A[0,0] = a
# A[1,1] = 2*a
# for ii in range(2,5):
#     A[ii,ii] = c
# A[5,5] = a
# for ii in range(6,16):
#     A[ii,ii] = 2*c 
# A[16,16] = a
# A[17,17] = 3*a   
# for ii in range(18,25):
#     A[ii,ii] = c
# A[25,25] = b
# for ii in range(26,31):
#     A[ii,ii] = 2*c
# A[31,31] = a


# A[0,0] = a
# for ii in range(1,31):
#     A[ii,ii] = 2*c  
# A[31,31] = b


# A[0,0] = a
# for ii in range(1,17):
#     A[ii,ii] = 2*c  
# for ii in range(17,29):
#     A[ii,ii] = 4*c  
# A[29,29] = 6*b
# A[30,30] = 6*b
# A[31,31] = b


A[0,0] = a
for ii in range(1,17):
    A[ii,ii] = 2*c  
for ii in range(17,29):
    A[ii,ii] = 4*c  
A[29,29] = 6*b
A[30,30] = 6*b
A[31,31] = b

# ---
A_norm = qucf_o.normalize_matrix_A(A, D)

# Define the circuit structure (problem-specific part):
oo_circ = qucf_o.Circuit__()
anc = qucf_o.Regs__()
inp = qucf_o.Regs__()
anc.add_reg("af", 1, True)
anc.add_reg("ar", 1, True)
inp.add_reg("f",  nf, False)
inp.add_reg("r",  nr, False)
oo_circ.set_regs(inp, anc)
oo_circ.compute_N_registers()

amin.(excl. zero) value in D: 		1.000e+00
amax. value in A: 			6.240e+01
amin.(excl. zero) value in A: 		1.200e+00

amax. value in A-norm: 			1.000e+00
amin. (excl. zero)  value in A-norm: 	1.923e-02
Adding a register 'af'...
Adding a register 'ar'...
Adding a register 'f'...
Adding a register 'r'...


In [42]:
# -----------------------------------------------
# --- Computation of oracle angles ---
# -----------------------------------------------
mix.reload_module(mix)
mix.reload_module(qucf_o)

# 1. Compute angles of rotations gates:
B = np.divide(A_norm, D, out=np.zeros_like(A), where = D!=0)
Ty, Tc_ampl, Tc_phase = qucf_o.compute_angles(B)

# 2. Rearrange angles:
grid_Ry       = qucf_o.create_grid_of_gates(oo_circ, Ty)
grid_Rc_ampl  = qucf_o.create_grid_of_gates(oo_circ, Tc_ampl)
grid_Rc_phase = qucf_o.create_grid_of_gates(oo_circ, Tc_phase)

# Create original groups:
# groups_Ry = qucf_o.create_groups(oo_circ, grid_Ry,      grid_R_phase=None,        )
# groups_Rc = qucf_o.create_groups(oo_circ, grid_Rc_ampl, grid_R_phase=grid_Rc_phase)

groups_Ry = qucf_o.create_groups_neighbor(oo_circ, grid_Ry,      grid_R_phase=None,        )
groups_Rc = qucf_o.create_groups_neighbor(oo_circ, grid_Rc_ampl, grid_R_phase=grid_Rc_phase)
sys_gates_ = qucf_o.SystemGates__(oo_circ, groups_Ry, groups_Rc)

# split groups:
sys_gates_.n_split_ = 1
sys_gates_.split_gates_groups()

# sort groups:
sys_gates_.sort_gates_groups()

# extend the gates:
sys_gates_.extend_gates_groups()

# 4 Construct the circuit:
sys_gates_.construct_circuit_OH(path_, file_name_oracle_)

N-input-registers = 2 with [1, 4] qubits
N-ancilla-registers = 2 with [1, 1] qubits
Total number of input qubits: 5
Matrix size that can be encoded: 32
N-gates: 12


In [43]:
# --- Recreate the matrix using the groups ---
mix.reload_module(qucf_o)
qucf_o.compare_matrices(oo_circ, sys_gates_.reconstruct_matrix_using_groups(), B)

# --- Count the number of groups ---
sys_gates_.count_groups()

Matrix size: 32

Max. abs. diff-real: 8.327e-17
Max. abs. diff-imag: 5.551e-17

N = 32
--- Groups for the gate Ry ---
Number of groups: 1
--- Groups for the gate Rc ---
Number of groups: 5



In [44]:
# ----------------------------------------------------------
# --- Read results ---
# ----------------------------------------------------------
mix.reload_module(qucf_r)
mix.reload_module(qucf_o)

dd_res = qucf_r.read_matrix(path_, "diag-matrix_OUTPUT.hdf5") 

B_res = np.divide(dd_res["A"], D, out=np.zeros_like(dd_res["A"]), where = D!=0)

qucf_o.compare_matrices(oo_circ, B_res, B)

Reading the matrix from: diag-matrix_OUTPUT.hdf5
from the path: ../simulations/tests/test-diag-matrix
date of the simulation:  07-03-2023 16:04:50
matrix name:  U_BE
N = 32

Max. abs. diff-real: 1.567e-13
Max. abs. diff-imag: 2.043e-13


In [ ]:
# --------------------------------------------------------------
# --- Plot matrix D and A-real---
# --------------------------------------------------------------
mix.reload_module(qucf_r)
figD     = qucf_r.plot_matrix(dd["A-mask"], fontsize = 20, cmap = "bwr")
figA_real = qucf_r.plot_matrix(np.real(A), fontsize = 20, cmap = "bwr")

In [10]:
# ---------------------------------------------------------------
# --- Function to print matrix subblocks ---
# ---------------------------------------------------------------
mix.reload_module(mix)
def print_submatrix(B, flag_imag = False):
    N = B.shape[0]

    
    print("\n")
    print("****** N: {:d} *************************************".format(N))
    
    max_val = np.max(np.max(np.abs(B)))
    print("max|A|: {:0.3e}".format(max_val))
    
    min_val = np.min(np.min(np.abs(B[np.nonzero(B)])))
    print("min|A|: {:0.3e}".format(min_val))
    
    # --------------------------------------------------
    Nr_print, Nc_print = N, N
    ir_start, ic_start = 0, 0
    
    B_plot = np.imag(B) if flag_imag else np.real(B)
    line_desr = "Imag part" if flag_imag else "Real part"


    print("-------------------------- {:s} -----------------------------------------".format(line_desr))
    mix.print_subblock_colored(
        B_plot, 
        ir_start, ic_start,
        Nr_print, Nc_print,
        ff=[6, 3, "f"], 
        n_in_row = Nc_print, 
        gap_be = " ", 
    )

In [16]:
# ------------------------------------------
# --- Print matrix D ---
# ------------------------------------------
print_submatrix(D) 



****** N: 32 *************************************
max|A|: 1.000e+00
min|A|: 1.000e+00
-------------------------- Real part -----------------------------------------
--- 0:   1.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 1:   0.000  1.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 2:   0.000  0.000  1.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 3:   0.000  0.000  0.000  1.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0

In [11]:
# ------------------------------------------
# --- Print matrix B = A_norm/D  ---
# ------------------------------------------
print_submatrix(B, False) 
print_submatrix(B, True) 

# print_submatrix(dd_res["A-real"], False) 
# print_submatrix(dd_res["A-imag"], False) 



****** N: 32 *************************************
max|A|: 1.000e+00
min|A|: 1.923e-02
-------------------------- Real part -----------------------------------------
--- 0:   0.019  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 1:   0.000  0.147  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 2:   0.000  0.000  0.147  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 3:   0.000  0.000  0.000  0.147  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0

In [24]:
theta_y = 2.434655880435e+00
print("{:0.3e}".format(np.cos(theta_y/2.)))

3.462e-01


In [26]:
theta_z, theta_y = -4.347141368359e-02, 2.845571544992e+00
print("{:20.3e}".format(np.exp(- 1j * theta_z/2.) * np.cos(theta_y/2.)))
# print("{:20.3e}".format(d_coef * np.cos(theta_y/2.)))

1.474e-01+3.205e-03j


In [4]:
def test_func(a, b):
    print(a)
    print(b)
    return

test_func([10, 20])

TypeError: test_func() missing 1 required positional argument: 'b'

In [6]:
xx = [
    [10, 20, 30, 40],
    [100, 200 ,300, 400]
]

xx1 = xx[1]

xx1[0] = 200
xx1.append(600)

print(xx)

[[10, 20, 30, 40], [200, 200, 300, 400, 600]]


In [7]:
xx = True
yy = not xx
print(yy)

False


In [12]:
class AA:
    x = None
    rr = None
    nr = None
    def __init__(self, x0):
        self.x = x0
        self.rr = [10, 20]
        self.nr = np.array([1,2,3])
        
oo1 = AA(10)
oo2 = AA(20)
xx = [oo1, oo2]

xx[0].x = 100
xx[0].rr = [100, 200, 300]
xx[1].nr[1] = 20

print(oo1.x)
print(oo1.rr)
print(oo1.nr)
print(oo2.nr)

100
[100, 200, 300]
[1 2 3]
[ 1 20  3]


In [13]:
def ff(xx):
    xx.append(100)
    
yy = [1,2,3,4]
ff(yy)

print(yy)

[1, 2, 3, 4, 100]


In [9]:
def get_data():
    return 10, 20

def target_f(x12, x3):
    x1, x2 = x12
    print(x1)
    print(x2)
    print(x3)
    
target_f(get_data(), 30)
print()
target_f((100, 200), 30)

10
20
30

100
200
30


In [16]:
xx = np.array([10, 20, 30, 40])

if 20 in xx:
    print("here")

here
